Notes:
-prob decreases as node increases

In [1]:
using LightGraphs

"""
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [2]:
"""
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;


"""
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;




In [3]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*5. 
b[1] = 5
genb = copy(b)  # save for later
g = randEdgeGen(g,10000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
push!(L, laplacian_matrix(g))
numnewnodes = 30
a_0 = -4
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes  
    g = addPrefNode(g,b, a_0)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-2+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
;

const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 2
lambda = 0.0001
new = numnewnodes
a = zeros(t-1,new)
b = zeros(t_0)
u = zeros(t-1,new)
iters = 0
diff = 1.0
b_old = b;


Current

In [6]:
@time begin

for j in 1:800 
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
   	end
        
    #b update
    c = zeros(t-1)
    for i in 1:new
        c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ (u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i]))/(rho*new)
    end
    b = soft(c[1:t_0],2*lambda/rho)
    for i in 1:new
    	u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end


end


0.020144972278299835
0.02014405866176236
0.020143145049503302
0.02014223144152998
0.020141317837851797
0.02014040423847642
0.020139490643411473
0.02013857705266587
0.02013766346624706
0.02013674988416393
0.020135836306424848
0.02013492273303654
0.0201340091640073
0.020133095599346753
0.02013218203906247
0.020131268483161976
0.020130354931655024
0.020129441384547763
0.02012852784184913
0.02012761430356753
0.020126700769712718
0.020125787240289437
0.020124873715309195
0.020123960194777628
0.020123046678704945
0.020122133167098665
0.020121219659967448
0.020120306157318565
0.020119392659161257
0.020118479165502308
0.020117565676352152
0.020116652191717502
0.02011573871160665


LoadError: InterruptException:

In [8]:
sort(b[:,1])

1023-element Array{Float64,1}:
 -1.09439 
 -1.06934 
 -1.02492 
 -1.011   
 -0.999319
 -0.986744
 -0.94665 
 -0.905016
 -0.89458 
 -0.871695
 -0.864714
 -0.86192 
 -0.855307
  ⋮       
  2.08744 
  2.16603 
  2.16905 
  2.25406 
  2.32196 
  2.3836  
  2.46459 
  2.88548 
  3.02834 
  3.24035 
  3.28126 
  3.73887 

In [7]:
sum(genb)

45.0

Old version

@time begin

for j in 1:50     
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:new
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end
    
end

sort(b[:,1])

Ver 2

@time begin

for j in 1:50 
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
   	end
        
    #b update
    c = zeros(t-1)
    for i in 1:new
    	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i])/(rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
    	u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end


end


Ver 3

b

@time begin

b = zeros(t-1)
b_old = b
for j in 1:50 
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,b[1:(t_0 +i-1)],u[1:(t_0 +i-1),i])
   	end
        
    #b update
    c = zeros(t-1)
    for i in 1:new
    	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i])/(rho)
    end
    b = soft(c,lambda)
    for i in 1:new
    	u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-b[1:(t_0 +i-1)])
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

@time begin
for j in 1:1000     
    a= newton(A,a_0,L,rho,b,u)
    #b update
    c = zeros(t-1)
    c = c+ u+rho*(L*a)/(rho)
    b = soft(c,lambda)
    u = u+ rho*(L*a-b)
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end
end
end
